In [1]:
import requests
from bs4 import BeautifulSoup
import pymongo
import time
from selenium import webdriver

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# News Article

In [3]:
# Define database and collection
db = client.mars_db
collection = db.articles

In [4]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
# Base URL for make relative links explicit
base_url = 'https://mars.nasa.gov'

In [5]:
# Retrieve page with the requests module
def render_page(url):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(3)
    response = driver.page_source
    #driver.quit()
    return response

In [6]:
response = render_page(url)

In [7]:
soup = BeautifulSoup(response, "html.parser")
#print(soup.prettify())

In [8]:
# Retrieve latest news article
news = soup.find('li', class_='slide')

title = news.find('div', class_='content_title').text
# Identify and return title
date = news.find('div', class_='list_date').text
# Identify and return link
link = news.a['href']
# Identify and return teaser text
text = news.find('div', class_='article_teaser_body').text
   
# Concat URL
concatURL = base_url + link

# Dictionary to be inserted as a MongoDB document
article = {
    'title': title,
    'date': date,
    'url': concatURL,
    'teaser': text
}

collection.insert_one(article)

# Feature Image

In [9]:
# Define database and collection
db = client.mars_db
collection = db.images

In [10]:
# URL of page to be scraped
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
# Base URL for make relative links explicit
image_base_url = 'https://www.jpl.nasa.gov'

In [11]:
# Retrieve page with the requests module
def render_page(image_url):
    driver = webdriver.Chrome()
    driver.get(image_url)
    time.sleep(3)
    image_response = driver.page_source
    #driver.quit()
    return image_response

In [12]:
image_response = render_page(image_url)

In [13]:
image_soup = BeautifulSoup(image_response, "html.parser")
#print(image_soup.prettify())

In [14]:
# Retrieve image result
image_results = image_soup.find('footer')

# Identify and return image title
img_title = image_results.a['data-title']
# Identify and return image src
img_link = image_results.a['data-fancybox-href']
     
# Concat URL
imageURL = image_base_url + img_link

# Dictionary to be inserted as a MongoDB document
image = {
    'imgTitle': img_title,
    'imgSrc': imageURL
}

collection.insert_one(image)

# Current Weather

In [15]:
# Define database and collection
db = client.mars_db
collection = db.weather

In [16]:
# URL of page to be scraped
weather_url = 'https://twitter.com/marswxreport?lang=en'

In [17]:
# Retrieve page with the requests module
def render_page(weather_url):
    driver = webdriver.Chrome()
    driver.get(weather_url)
    time.sleep(3)
    weather_response = driver.page_source
    #driver.quit()
    return weather_response

In [18]:
weather_response = render_page(weather_url)

In [19]:
weathersoup = BeautifulSoup(weather_response, "html.parser")
#print(weathersoup.prettify())

In [20]:
# Get Current Weather
weather = weathersoup.find('div', class_='js-tweet-text-container')

mars_weather = weathersoup.find('p', class_='tweet-text').text

# Dictionary to be inserted as a MongoDB document
weather = {
    'weather': mars_weather
}

collection.insert_one(weather)

# Query MongoDB

In [21]:
# Display news article in MongoDB collection
articles = db.articles.find()

for article in articles:
    print(article)

{'_id': ObjectId('5be5f4ef755db92546e34d73'), 'title': 'Curiosity on the Move Again', 'date': 'November  6, 2018', 'url': 'https://mars.nasa.gov/news/8371/curiosity-on-the-move-again/', 'teaser': "NASA's Mars Curiosity rover drove about 197 feet over the weekend to a site called Lake Orcadie, pushing its total odometry to over 12 miles."}


In [22]:
# Display featured image in MongoDB collection
images = db.images.find()

for image in images:
    print(image)

{'_id': ObjectId('5be5f4f6755db92546e34d74'), 'imgTitle': "NASA's Orbiting Carbon Observatory (OCO)-2 (Artist Concept)", 'imgSrc': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17896_ip.jpg'}


In [23]:
# Display current weather in MongoDB collection
weather = db.weather.find()

for weather in weather:
    print(weather)

{'_id': ObjectId('5be5f503755db92546e34d75'), 'weather': 'Sol 2223 (2018-11-07), high 2C/35F, low -64C/-83F, pressure at 8.50 hPa, daylight 06:18-18:35'}
